# ***Imports***

In [ ]:
!pip install PyPDF2
!pip install pdfplumber
!pip install flair
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Install pdfminer.six if you haven't already
!pip install pdfminer.six
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 86.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
from pdfminer.high_level import extract_text
import spacy
import re

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
import re
from pdfminer.high_level import extract_text  # Assuming you have pdfminer installed

# Load the NER tagger
tagger = SequenceTagger.load('ner')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/419M [00:00<?, ?B/s]

2024-11-26 15:06:38,486 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


# ***Extract full text from cv***

In [ ]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

# ***Get full name***

In [ ]:
# Function to extract full name using Flair
def extract_full_name_with_flair(text):
    top_of_cv = text[:500]
    sentence = Sentence(top_of_cv)
    tagger.predict(sentence)

    # Extract PERSON entities from the tagged sentence
    name_entities = []
    for entity in sentence.get_spans('ner'):
        if entity.get_label("ner").value == "PER":
            name_entities.append(entity.text)

    if name_entities:
        return " ".join(name_entities)

    # Fallback to regex if Flair doesn't find a name
    name_regex = r"([A-Z][a-z]+(?:\s+[A-Z][a-z]+)+)"
    name_matches = re.findall(name_regex, top_of_cv)

    if name_matches:
        return name_matches[0]

    return "Name not found."

# ***References***

In [ ]:
# Function to extract contact info
def extract_contact_info(text):
    possible_reference_keywords = ["references", "referees", "non related references"]
    reference_section_pos = len(text)

    # Locate the references section (if it exists)
    for keyword in possible_reference_keywords:
        pos = text.lower().find(keyword)
        if pos != -1 and pos < reference_section_pos:
            reference_section_pos = pos

    # Search for contact info before the references section
    contact_text = text[:reference_section_pos]

    # Regular expressions for phone and email
    phone_regex = r"(\+?\d{1,4}[\s-]?)?(\d{9,10})"
    email_regex = r"([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"

    # Extract phone and email
    phone_matches = re.findall(phone_regex, contact_text)
    email_matches = re.findall(email_regex, contact_text)

    # Get the first phone and email (likely belonging to the CV owner)
    phone = phone_matches[0][1] if phone_matches else None
    email = email_matches[0] if email_matches else None

    contact_info = f"Phone: {phone}, Email: {email}"
    return contact_info if phone or email else "Contact Info not found."

# ***Other sections we are extracting from the text***

In [ ]:
# Define alternative keywords for sections
section_keywords = {
    "Job Role": ["Job Role", "Position", "Current Position", "Objective", "Professional Summary"],
    "Qualifications": ["Qualifications", "Education", "Academic Background"],
    "Experiences": ["Experiences", "Experience", "Professional Experience", "Work History", "Employment"],
    "Address": ["Address", "Location", "Residence"],
    "Skills": ["Skills", "Technical Skills", "Core Competencies"],
    "Projects": ["Projects", "Project Work", "Key Projects", "Research Projects"],
    "References": ["References", "Referees", "Non related References"]
}

In [ ]:
# Function to dynamically extract a specific section
def extract_section(text, section_name):
    text_lower = text.lower()

    section_start = -1
    for keyword in section_keywords[section_name]:
        keyword_lower = keyword.lower()
        section_start = text_lower.find(keyword_lower)
        if section_start != -1:
            break

    if section_start == -1:
        return f"{section_name} section not found."

    next_section_start = len(text)
    for key, alternatives in section_keywords.items():
        if key != section_name:
            for alt_keyword in alternatives:
                keyword_start = text_lower.find(alt_keyword.lower(), section_start + len(keyword_lower))
                if keyword_start != -1 and keyword_start < next_section_start:
                    next_section_start = keyword_start

    section_content = text[section_start:next_section_start].strip()
    return section_content

# ***Assigning possible job roles for the analysis***

In [ ]:
# Define a mapping of job roles to skills and project-related keywords
job_role_keywords = {
    "Software Developer": [ "programming", "python", "java", "c++", "git", "version control", "algorithms",
    "data structures", "object-oriented", "testing", "debugging", "problem-solving",
    "api", "full stack", "backend", "frontend"],

    "Data Scientist": [ "machine learning", "data analysis", "python", "r", "statistics", "data mining",
    "modeling", "big data", "data visualization", "deep learning", "pandas", "numpy",
    "matplotlib", "neural networks", "artificial intelligence", "regression",
    "classification", "clustering"],

    "Web Developer": ["html", "css", "javascript", "web", "react", "angular", "frontend", "backend",
    "php", "sql", "mysql", "bootstrap", "node.js", "express", "ui/ux", "web design",
    "responsive design", "jquery", "ajax", "api integration"],

    "Mobile App Developer": [ "android", "ios", "flutter", "react native", "kotlin", "swift", "mobile app",
    "development", "mobile ui", "mobile ux", "cross-platform", "native app",
    "app store", "google play", "firebase", "push notifications", "in-app purchases"],

    "Project Manager": [ "leadership", "project management", "team", "planning", "agile", "scrum", "jira",
    "kanban", "waterfall", "communication", "risk management", "budgeting",
    "stakeholder management", "scheduling", "milestones", "resource allocation",
    "time management", "project charter", "gantt chart"],

    "DevOps Engineer": ["docker", "kubernetes", "ci/cd", "automation", "aws", "azure", "cloud",
    "infrastructure", "terraform", "ansible", "jenkins", "linux", "bash scripting",
    "monitoring", "logging", "scaling", "containers", "orchestration", "security",
    "microservices", "networking"]
}

In [ ]:
# Function to determine all possible job roles based on skills and projects
def determine_possible_job_roles(skills_text, projects_text):
    combined_text = f"{skills_text} {projects_text}".lower()  # Combine and convert to lowercase for matching

    matched_roles = {}

    # Check for keyword matches in both skills and projects
    for role, keywords in job_role_keywords.items():
        for keyword in keywords:
            if keyword in combined_text:
                matched_roles[role] = matched_roles.get(role, 0) + 1  # Count the keyword match for ranking

    # Sort roles by the number of keyword matches
    if matched_roles:
        sorted_roles = sorted(matched_roles.items(), key=lambda x: x[1], reverse=True)
        return [(role, count) for role, count in sorted_roles]  # Return all roles with match counts

    return [("No matching job roles found.", 0)]

# ***Cv paths***

In [ ]:
# List of PDF paths
pdf_paths = [
    "/content/drive/My Drive/IIT Degree/IIT esandu Obadaarachchi_CV.pdf",
    "/content/drive/My Drive/IIT Degree/haritha.pdf",
    "/content/drive/My Drive/IIT Degree/kohul.pdf",
    "/content/drive/My Drive/IIT Degree/sakindu.pdf",
    "/content/drive/My Drive/IIT Degree/arkashi.pdf",
]

# ***map to store all data***

In [ ]:
# Map to store details from all CVs
all_cv_details = {}


In [ ]:
# Loop through each PDF and extract details
for pdf_path in pdf_paths:
    extracted_text = extract_text_from_pdf(pdf_path)

    # Extracting various fields
    full_name = extract_full_name_with_flair(extracted_text)
    contact_info = extract_contact_info(extracted_text)
    job_role = extract_section(extracted_text, "Job Role")
    qualifications = extract_section(extracted_text, "Qualifications")
    experiences = extract_section(extracted_text, "Experiences")
    address = extract_section(extracted_text, "Address")
    skills = extract_section(extracted_text, "Skills")
    projects = extract_section(extracted_text, "Projects")
    references = extract_section(extracted_text, "References")

    # Determine possible job roles based on extracted skills and projects
    possible_job_roles = determine_possible_job_roles(skills, projects)

    # Store extracted details in the map (dictionary)
    extracted_details = {
        "Full Name": full_name,
        "Contact Info": contact_info,
        "Job Role": job_role,
        "Qualifications": qualifications,
        "Experiences": experiences,
        "Address": address,
        "Skills": skills,
        "Projects": projects,
        "References": references,
        "Possible Job Roles": [f"{role} (Matched Keywords: {count})" for role, count in possible_job_roles]
    }

    # Use the PDF path as the key to store the details in the map
    all_cv_details[pdf_path] = extracted_details

# Print all extracted details
for path, details in all_cv_details.items():
    print(f"Details for {path}:\n")
    for key, value in details.items():
        print(f"{key}: {value}\n")
        print("------------------------NEXT-----------------------\n")
    print("-------------------------------------------------\n")


Details for /content/drive/My Drive/IIT Degree/IIT esandu Obadaarachchi_CV.pdf:

Full Name: Don Esandu Meth Obadaaarachchi

------------------------NEXT-----------------------

Contact Info: Phone: 767790567, Email: eobadaarachchi@gmail.com

------------------------NEXT-----------------------

Job Role: Job Role section not found.

------------------------NEXT-----------------------

Qualifications: Education

Bachelor of Science (Hons) in Artificial Intelligence and Data Science 

2022 – Present

Informatics Institute of Technology affiliated to Robert Gordon University, UK
 Results= Computational Mathematics - (B) , Programming Fundamentals - (B) 

        Computer System Fundamentals - (A) , DataBase Systems - (B) , Structures and Algorithms - (A) , 
        Web Development - (A) , Artificial Intelligence - (A), Advance Mathematics for Data Science - (A),  
        Object Oriented Programming - (B)  

   Cambridge A Levels Science Stream 

Lyceum International School Nugegoda, Sri L

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Combine all sections from the extracted text into one
def combine_cv_sections(cv_details):
    sections = [
        #cv_details.get("Full Name", ""),
        cv_details.get("Job Role", ""),
        cv_details.get("Qualifications", ""),
        cv_details.get("Experiences", ""),
        cv_details.get("Skills", ""),
        cv_details.get("Projects", ""),
    ]
    return " ".join(sections)

# Combine all CVs into a list of strings
cv_texts = [combine_cv_sections(details) for details in all_cv_details.values()]

# User Prompt
user_prompt = "LOOKING for a person who has done ai data science , machine learning projects"

# Vectorize CVs and User Prompt using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_features=10000)  # Limit to top 5000 features
cv_tfidf_matrix = tfidf_vectorizer.fit_transform(cv_texts)  # Fit and transform CVs
user_prompt_tfidf = tfidf_vectorizer.transform([user_prompt])  # Transform user prompt

# Calculate cosine similarity between user prompt and each CV
similarities = cosine_similarity(user_prompt_tfidf, cv_tfidf_matrix)[0]  # Returns a 1D array of similarities

# Rank CVs by similarity score
ranked_indices = np.argsort(similarities)[::-1]  # Sort in descending order
ranked_cvs = [(list(all_cv_details.keys())[i], similarities[i]) for i in ranked_indices]

# Display ranked CVs
print("Ranked CVs based on similarity to user prompt:")
for rank, (cv_path, similarity) in enumerate(ranked_cvs, 1):
    print(f"Rank {rank}: {cv_path} (Similarity Score: {similarity:.2f})")

Ranked CVs based on similarity to user prompt:
Rank 1: /content/drive/My Drive/IIT Degree/arkashi.pdf (Similarity Score: 0.23)
Rank 2: /content/drive/My Drive/IIT Degree/IIT esandu Obadaarachchi_CV.pdf (Similarity Score: 0.13)
Rank 3: /content/drive/My Drive/IIT Degree/sakindu.pdf (Similarity Score: 0.07)
Rank 4: /content/drive/My Drive/IIT Degree/kohul.pdf (Similarity Score: 0.03)
Rank 5: /content/drive/My Drive/IIT Degree/haritha.pdf (Similarity Score: 0.01)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Map to store details from all CVs
all_cv_details = {}

# List to store raw extracted text for vectorization
raw_texts = []

# Loop through each PDF and extract details
for pdf_path in pdf_paths:
    extracted_text = extract_text_from_pdf(pdf_path)
    raw_texts.append(extracted_text)  # Store the entire raw text for vectorization

    # Extracting various fields
    full_name = extract_full_name_with_flair(extracted_text)
    contact_info = extract_contact_info(extracted_text)
    job_role = extract_section(extracted_text, "Job Role")
    qualifications = extract_section(extracted_text, "Qualifications")
    experiences = extract_section(extracted_text, "Experiences")
    address = extract_section(extracted_text, "Address")
    skills = extract_section(extracted_text, "Skills")
    projects = extract_section(extracted_text, "Projects")
    references = extract_section(extracted_text, "References")

    # Determine possible job roles based on extracted skills and projects
    possible_job_roles = determine_possible_job_roles(skills, projects)

    # Store extracted details in the map (dictionary)
    extracted_details = {
        "Full Name": full_name,
        "Contact Info": contact_info,
        "Job Role": job_role,
        "Qualifications": qualifications,
        "Experiences": experiences,
        "Address": address,
        "Skills": skills,
        "Projects": projects,
        "References": references,
        "Raw Text": extracted_text,  # Add the entire raw text to the dictionary
        "Possible Job Roles": [f"{role} (Matched Keywords: {count})" for role, count in possible_job_roles]
    }

    # Use the PDF path as the key to store the details in the map
    all_cv_details[pdf_path] = extracted_details


Ranked CVs based on similarity to user prompt:
Rank 1: /content/drive/My Drive/IIT Degree/arkashi.pdf (Similarity Score: 0.22)
Rank 2: /content/drive/My Drive/IIT Degree/IIT esandu Obadaarachchi_CV.pdf (Similarity Score: 0.12)
Rank 3: /content/drive/My Drive/IIT Degree/sakindu.pdf (Similarity Score: 0.08)
Rank 4: /content/drive/My Drive/IIT Degree/haritha.pdf (Similarity Score: 0.01)
Rank 5: /content/drive/My Drive/IIT Degree/kohul.pdf (Similarity Score: 0.00)

Most Relevant CV (/content/drive/My Drive/IIT Degree/arkashi.pdf):

RAMANAYAKE ARACHCHIGE DONA
CHETHANA AKARSHANI

Bsc. (Hons) Artificial Intelligence  and Data Science

Date of birth: 20/08/2000 

      |       phone number: 0719993444

|         Instagram: akarshani_r

|      Email: akarshaniramanayake0820@gmail.com       |

LinkedIn: www.linkedin.com/in/akarshani

Experience

Assistant lecturer (Full-time)
Informatics Institute of Technology

Aug 2024- Present 

Assigned  to  do  tutorials  on  computer  systems  fundamentals

In [ ]:

# User Prompt
user_prompt = "LOOKING for a person who has done ai data science , machine learning projects"

# Vectorize CVs and User Prompt using TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words="english", max_features=10)  # Limit to top features
cv_tfidf_matrix = tfidf_vectorizer.fit_transform(raw_texts)  # Fit and transform raw texts
user_prompt_tfidf = tfidf_vectorizer.transform([user_prompt])  # Transform user prompt

# Calculate cosine similarity between user prompt and each CV
similarities = cosine_similarity(user_prompt_tfidf, cv_tfidf_matrix)[0]  # Returns a 1D array of similarities

# Rank CVs by similarity score
ranked_indices = np.argsort(similarities)[::-1]  # Sort in descending order
ranked_cvs = [(list(all_cv_details.keys())[i], similarities[i]) for i in ranked_indices]

# Display ranked CVs
print("Ranked CVs based on similarity to user prompt:")
for rank, (cv_path, similarity) in enumerate(ranked_cvs, 1):
    print(f"Rank {rank}: {cv_path} (Similarity Score: {similarity:.2f})")

# Optional: Display the most relevant CV's raw text
if ranked_cvs:
    most_relevant_cv_path = ranked_cvs[0][0]
    print(f"\nMost Relevant CV ({most_relevant_cv_path}):\n")
    print(all_cv_details[most_relevant_cv_path]["Raw Text"])

Ranked CVs based on similarity to user prompt:
Rank 1: /content/drive/My Drive/IIT Degree/arkashi.pdf (Similarity Score: 0.69)
Rank 2: /content/drive/My Drive/IIT Degree/IIT esandu Obadaarachchi_CV.pdf (Similarity Score: 0.41)
Rank 3: /content/drive/My Drive/IIT Degree/sakindu.pdf (Similarity Score: 0.05)
Rank 4: /content/drive/My Drive/IIT Degree/kohul.pdf (Similarity Score: 0.00)
Rank 5: /content/drive/My Drive/IIT Degree/haritha.pdf (Similarity Score: 0.00)

Most Relevant CV (/content/drive/My Drive/IIT Degree/arkashi.pdf):

RAMANAYAKE ARACHCHIGE DONA
CHETHANA AKARSHANI

Bsc. (Hons) Artificial Intelligence  and Data Science

Date of birth: 20/08/2000 

      |       phone number: 0719993444

|         Instagram: akarshani_r

|      Email: akarshaniramanayake0820@gmail.com       |

LinkedIn: www.linkedin.com/in/akarshani

Experience

Assistant lecturer (Full-time)
Informatics Institute of Technology

Aug 2024- Present 

Assigned  to  do  tutorials  on  computer  systems  fundamentals